Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (197738, 28, 28) (197738,)
Validation set (9847, 28, 28) (9847,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197738, 784) (197738, 10)
Validation set (9847, 784) (9847, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
lambda_term = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss+tf.reduce_mean(lambda_term*tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 45.797607
Training accuracy: 7.4%
Validation accuracy: 9.5%
Loss at step 100: 11.440479
Training accuracy: 73.9%
Validation accuracy: 72.8%
Loss at step 200: 4.351262
Training accuracy: 79.2%
Validation accuracy: 77.0%
Loss at step 300: 1.937797
Training accuracy: 82.6%
Validation accuracy: 79.8%
Loss at step 400: 1.116209
Training accuracy: 84.0%
Validation accuracy: 80.9%
Loss at step 500: 0.833416
Training accuracy: 84.5%
Validation accuracy: 81.6%
Loss at step 600: 0.734916
Training accuracy: 84.7%
Validation accuracy: 81.9%
Loss at step 700: 0.700282
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 800: 0.688008
Training accuracy: 84.8%
Validation accuracy: 82.1%
Test accuracy: 89.0%


In [7]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
lambda_term = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 45.849087
Minibatch accuracy: 9.4%
Validation accuracy: 13.6%
Minibatch loss at step 500: 0.998088
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 0.933491
Minibatch accuracy: 78.9%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 0.555281
Minibatch accuracy: 85.2%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.850815
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 0.739740
Minibatch accuracy: 80.5%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 0.626983
Minibatch accuracy: 84.4%
Validation accuracy: 81.2%
Test accuracy: 88.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

- Logistic Regression

In [9]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
lambda_term = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss+tf.reduce_mean(lambda_term*tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
num_steps = 801

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 45.201584
Training accuracy: 12.0%
Validation accuracy: 12.7%
Loss at step 100: 10.961988
Training accuracy: 100.0%
Validation accuracy: 44.2%
Loss at step 200: 4.087389
Training accuracy: 100.0%
Validation accuracy: 50.1%
Loss at step 300: 1.575105
Training accuracy: 100.0%
Validation accuracy: 55.4%
Loss at step 400: 0.659657
Training accuracy: 100.0%
Validation accuracy: 59.7%
Loss at step 500: 0.326426
Training accuracy: 100.0%
Validation accuracy: 62.3%
Loss at step 600: 0.204937
Training accuracy: 100.0%
Validation accuracy: 63.5%
Loss at step 700: 0.160541
Training accuracy: 100.0%
Validation accuracy: 64.1%
Loss at step 800: 0.144263
Training accuracy: 100.0%
Validation accuracy: 64.4%
Test accuracy: 71.3%


- Neural Networks

In [11]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
batch_size = 128
lambda_term = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 3001
train_dataset_2 = train_dataset[:1000]
train_labels_2 = train_labels[:1000]
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_2[offset:(offset + batch_size), :]
    batch_labels = train_labels_2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.569466
Minibatch accuracy: 7.8%
Validation accuracy: 10.7%
Minibatch loss at step 500: 0.578079
Minibatch accuracy: 97.7%
Validation accuracy: 77.6%
Minibatch loss at step 1000: 0.457415
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 0.419943
Minibatch accuracy: 96.9%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 0.471033
Minibatch accuracy: 95.3%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 0.632765
Minibatch accuracy: 89.8%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 0.409804
Minibatch accuracy: 97.7%
Validation accuracy: 78.4%
Test accuracy: 85.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [13]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
batch_size = 128
lambda_term = 0.01
num_hidden_nodes = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights_2 = tf.Variable(
        tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  keep_prob = tf.placeholder(tf.float32)

  def applyModelWithDataset(dataset):
    # Training computation.
    logits = tf.matmul(dataset, weights) + biases
    # hidden layer, of shape batch_size * num_hidden_nodes
    h_fc1_drop = tf.nn.relu(logits)
    h_fc1_drop = tf.nn.dropout(h_fc1_drop, keep_prob)
    logits = tf.matmul(h_fc1_drop, weights_2) + biases_2
    return logits

  logits = applyModelWithDataset(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*(tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_relu, weights_2) + biases_2)
  test_prediction_relu =  tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction_relu, weights_2) + biases_2)

In [14]:
num_steps = 3001
train_dataset_2 = train_dataset[:1000]
train_labels_2 = train_labels[:1000]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_2[offset:(offset + batch_size), :]
    batch_labels = train_labels_2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3605.321045
Minibatch accuracy: 9.4%
Validation accuracy: 22.5%
Minibatch loss at step 500: 21.233738
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 0.638227
Minibatch accuracy: 97.7%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 0.441318
Minibatch accuracy: 99.2%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 0.425680
Minibatch accuracy: 98.4%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 0.509790
Minibatch accuracy: 95.3%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 0.403134
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Test accuracy: 88.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


- 2 hidden layer (size=1024) w/ lambda = 0.001 w/ dropout

In [15]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
lambda_term = 0.001
num_hidden_nodes = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  keep_prob = tf.placeholder(tf.float32)

  # Training computation.
  logits = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden), keep_prob)
  logits = tf.matmul(logits, weights) + biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights) + biases)

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.9}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 680.027283
Minibatch accuracy: 9.4%
Validation accuracy: 31.5%
Minibatch loss at step 500: 206.151230
Minibatch accuracy: 78.1%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 116.161446
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 68.287697
Minibatch accuracy: 88.3%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 41.894054
Minibatch accuracy: 78.1%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.289301
Minibatch accuracy: 83.6%
Validation accuracy: 85.3%
Minibatch loss at step 3000: 15.340223
Minibatch accuracy: 89.1%
Validation accuracy: 86.3%
Test accuracy: 93.1%


- same thing as above, but lowering the lambda

In [17]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
lambda_term = 0.00001
num_hidden_nodes = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  keep_prob = tf.placeholder(tf.float32)

  # Training computation.
  logits = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden), keep_prob)
  logits = tf.matmul(logits, weights) + biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights) + biases)

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.9}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 359.588776
Minibatch accuracy: 11.7%
Validation accuracy: 26.5%
Minibatch loss at step 500: 28.454397
Minibatch accuracy: 78.1%
Validation accuracy: 77.7%
Minibatch loss at step 1000: 14.581084
Minibatch accuracy: 76.6%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 10.799992
Minibatch accuracy: 85.2%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 8.123363
Minibatch accuracy: 82.0%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 5.711651
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 7.397964
Minibatch accuracy: 82.0%
Validation accuracy: 81.0%
Test accuracy: 86.5%


In [19]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
lambda_term = 0.00001
num_hidden_nodes = 2048

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_hidden = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  keep_prob = tf.placeholder(tf.float32)

  # Training computation.
  logits = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden), keep_prob)
  logits = tf.matmul(logits, weights) + biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden)))

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights) + biases)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.9}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 623.773254
Minibatch accuracy: 9.4%
Validation accuracy: 25.8%
Minibatch loss at step 500: 53.856163
Minibatch accuracy: 79.7%
Validation accuracy: 82.5%
Minibatch loss at step 1000: 47.786446
Minibatch accuracy: 77.3%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 21.913746
Minibatch accuracy: 85.9%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 33.206390
Minibatch accuracy: 82.0%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 19.972513
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 12.551636
Minibatch accuracy: 89.1%
Validation accuracy: 85.2%
Test accuracy: 89.8%


In [21]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
lambda_term = 0.00001
num_hidden_nodes = 2048
num_hidden_nodes_l2 = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.get_variable("weights_hidden", [image_size * image_size, num_hidden_nodes], initializer=tf.contrib.layers.xavier_initializer())
  biases_hidden = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights_hidden_l2 = tf.get_variable("weights_hidden_l2", [num_hidden_nodes, num_hidden_nodes_l2], initializer=tf.contrib.layers.xavier_initializer())
  biases_hidden_l2 = tf.Variable(tf.zeros([num_hidden_nodes_l2]))
  
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes_l2, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  keep_prob = tf.placeholder(tf.float32)

  # Training computation.
  logits = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden), keep_prob)
  logits = tf.nn.dropout(tf.nn.relu(tf.matmul(logits, weights_hidden_l2) + biases_hidden_l2), keep_prob)
  logits = tf.matmul(logits, weights) + biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_hidden_l2)))

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_hidden_l2) + biases_hidden_l2
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(valid_prediction), weights) + biases)
  test_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden ), weights_hidden_l2) + biases_hidden_l2
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(test_prediction), weights) + biases)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.9}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9.897906
Minibatch accuracy: 14.1%
Validation accuracy: 19.5%
Minibatch loss at step 500: 0.608618
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Minibatch loss at step 1000: 0.581701
Minibatch accuracy: 86.7%
Validation accuracy: 86.8%
Minibatch loss at step 1500: 0.372432
Minibatch accuracy: 89.8%
Validation accuracy: 87.7%
Minibatch loss at step 2000: 0.570076
Minibatch accuracy: 85.9%
Validation accuracy: 88.2%
Minibatch loss at step 2500: 0.342896
Minibatch accuracy: 90.6%
Validation accuracy: 88.6%
Minibatch loss at step 3000: 0.320238
Minibatch accuracy: 92.2%
Validation accuracy: 88.7%
Test accuracy: 94.7%


In [25]:
# This is for neural networks

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
batch_size = 128
lambda_term = 0.00001
num_hidden_nodes = 2048
num_hidden_nodes_l2 = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden = tf.get_variable("weights_hidden", [image_size * image_size, num_hidden_nodes], initializer=tf.contrib.layers.xavier_initializer())
  biases_hidden = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  weights_hidden_l2 = tf.get_variable("weights_hidden_l2", [num_hidden_nodes, num_hidden_nodes_l2], initializer=tf.contrib.layers.xavier_initializer())
  biases_hidden_l2 = tf.Variable(tf.zeros([num_hidden_nodes_l2]))
  
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes_l2, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  keep_prob = tf.placeholder(tf.float32)

  # Training computation.
  logits = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden), keep_prob)
  logits = tf.nn.relu(tf.matmul(logits, weights_hidden_l2) + biases_hidden_l2)
  logits = tf.matmul(logits, weights) + biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss+lambda_term*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_hidden_l2)))

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_hidden_l2) + biases_hidden_l2
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(valid_prediction), weights) + biases)
  test_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden), weights_hidden_l2) + biases_hidden_l2
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(test_prediction), weights) + biases)

In [26]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.9}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 8.975744
Minibatch accuracy: 10.9%
Validation accuracy: 23.4%
Minibatch loss at step 500: 0.636498
Minibatch accuracy: 82.0%
Validation accuracy: 85.2%
Minibatch loss at step 1000: 0.576855
Minibatch accuracy: 83.6%
Validation accuracy: 86.9%
Minibatch loss at step 1500: 0.334463
Minibatch accuracy: 92.2%
Validation accuracy: 87.4%
Minibatch loss at step 2000: 0.558282
Minibatch accuracy: 84.4%
Validation accuracy: 88.0%
Minibatch loss at step 2500: 0.392018
Minibatch accuracy: 92.2%
Validation accuracy: 88.7%
Minibatch loss at step 3000: 0.258393
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 3500: 0.344863
Minibatch accuracy: 89.8%
Validation accuracy: 89.1%
Minibatch loss at step 4000: 0.370627
Minibatch accuracy: 89.1%
Validation accuracy: 88.8%
Minibatch loss at step 4500: 0.312529
Minibatch accuracy: 92.2%
Validation accuracy: 89.3%
Minibatch loss at step 5000: 0.280502
Minibatch accuracy: 94.5%
Validation accurac